# Injuries in Collision Prediction

In [1]:
import warnings
import pandas as pd
import sklearn
warnings.filterwarnings('ignore')

## Read data
+ run the collision_model_features.py file to get three data sets.
    + new_collisions
    + traffic_byhour
    + traffic_bydayofweek
+ read the three table to this notebook

In [2]:
path = '../matched_data/crash_matched_data'
collisions = pd.read_csv(
    path + '/new_collisions.csv')
traffic_byhour = pd.read_csv(
    path + '/traffic_byhour.csv')
traffic_bydayofweek = pd.read_csv(
    path + '/traffic_bydayofweek.csv')
print(collisions.columns)
print(traffic_byhour.columns)
print(traffic_bydayofweek.columns)

Index(['Unnamed: 0', 'collision_id', 'CRASH DATE', 'CRASH TIME', 'BOROUGH',
       'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME',
       'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'coords', 'index_right', 'Roadway.Name', 'From', 'To', 'int1', 'int2',
       'zip1', 'zip2', 'll1', 'll2', 'streetID', 'lat1', 'long1', 'lat2',
       'long2', 'dayofweek', 'time', 'speed_bumps', 'speed_li

In [7]:
# merge collisions dataset and two traffic datasets
# there may be missing values after joining, because only limited traffic volumes were only recorded on limited days
collisions = pd.merge(left=collisions, right=traffic_bydayofweek, how='left', on=['streetID','dayofweek'])
collisions = pd.merge(left=collisions, right=traffic_byhour, how='left', on=['streetID', 'time'])